# **Install dependencies**

In [1]:
pip install langchain langchain-community googlemaps gmaps openai gmplot python-dotenv torch jupyter_bokeh

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.4

In [14]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=87718d1c1d935fd477a1d66db2c207b3a6fd0d7fbcf9d992a14469d0a9ffabc2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


# Importación de GoogleMaps + OpenAI + HuggingFace

In [4]:
OpenAI_Key = 'sk-'
google_maps_API_Key = ''
hugging_face_tk= 'hf_'

# **Routing**

Include here all the necesary imports

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.agents import tool
from langchain.tools.render import format_tool_to_openai_function
import requests
from pydantic import BaseModel, Field
import googlemaps
import datetime
from typing import Optional

## **Langchain functions**

In [6]:
def find_place_return_address(google_maps_API_Key, input_text):

    gmaps = googlemaps.Client(key=google_maps_API_Key)

    result = gmaps.find_place(
        input=input_text,
        input_type='textquery',
        fields=[
            'formatted_address', 'geometry', 'types', 'place_id'
        ],
        language='es'
    )
    return result

In [7]:
# Define the GetCoordinates schema

class GetCoordinatesInput(BaseModel):
    place: str = Field(..., description="Place or address to get coordinates")
    google_api_key: str = Field(..., description="Google API Key")

@tool(args_schema=GetCoordinatesInput)
def get_coordinates(place: str, google_api_key: str) -> dict:
    """Get the coordinates of an address"""
    gmaps = googlemaps.Client(key=google_api_key)
    # Hacer la solicitud geocoding, devuelve la latitude, longitude...
    address = find_place_return_address(google_maps_API_Key, place)['candidates'][0]['formatted_address']
    geocode_result = gmaps.geocode(address)

    # Extraer las coordenadas (latitud y longitud)
    location = geocode_result[0]['geometry']['location']
    text_result = f'Address: {address} \nLatitud: {location["lat"]}, Longitud: {location["lng"]}'
    result = {'address': address,'location': location, 'text_result': text_result}
    return {'location': location, 'text_result': text_result}

In [8]:
# Define GetAddressFromCoordinates

class GetAddressFromCoordinatesInput(BaseModel):
    latitude: float = Field(..., description="Latitude")
    longitude: float = Field(..., description="Longitude")
    google_api_key: str = Field(..., description="Google API Key")

@tool(args_schema=GetAddressFromCoordinatesInput)
def get_address_from_coordinates(latitude: float, longitude: float, google_api_key: str) -> dict:
    """Get the address from coordinates"""
    gmaps = googlemaps.Client(key=google_api_key)
    reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))

    formated_address = reverse_geocode_result[0]["formatted_address"]
    text_result = f'Latitud: {latitude}, Longitud: {longitude} \nAddress: {formated_address}'
    result = {'address': formated_address,'location': {'lat': latitude, 'lng': longitude}, 'text_result': text_result}
    return result

In [11]:
# Define GetDirections
class GetDirectionsInput(BaseModel):
    origin: str = Field(..., description="Origin place/address")
    destination: str = Field(..., description="Destination place/address")
    mode: Optional[str] = Field(None, description="Travel mode (driving, walking, bicycling, transit)")
    language: str = Field(None, description="Language code (e.g., 'es', 'en')")
    google_api_key: str = Field(..., description="Google API Key")


@tool(args_schema=GetDirectionsInput)
def get_directions(origin: str, destination: str, google_api_key: str, mode: Optional[str] = 'driving',
                   language: str = 'es') -> dict:
    """Get directions from origin to destination"""
    # Initialize Google Maps client with the API key
    gmaps = googlemaps.Client(key=google_api_key)

    # Get the current date and time
    now = datetime.datetime.now()

    origin = find_place_return_address(google_maps_API_Key, origin)['candidates'][0]['formatted_address']
    destination = find_place_return_address(google_maps_API_Key, destination)['candidates'][0]['formatted_address']

    # Request directions between origin and destination
    directions_result = gmaps.directions(
        origin=origin,
        destination=destination,
        mode=mode,  # mode: ["driving", "walking", "bicycling", "transit"]
        departure_time=now,
        language=language,
        traffic_model='best_guess'  # traffic_model: ["best_guess", "optimistic", "pessimistic"]
    )

    # Extract relevant information
    if directions_result:
        route = directions_result[0]['legs'][0]
        distance_total = route['distance']['text']
        duration_total = route['duration']['text']
        steps = route['steps']
        directions = ''

        # Extract step-by-step instructions and distances
        for i, step in enumerate(steps):
            # Remove HTML tags from the instructions
            instructions = step['html_instructions'].replace('<b>', '').replace('</b>', '').replace('<div style="font-size:0.9em">', ' ').replace('</div>', '')
            distance = step['distance']['text']
            directions += f"{i + 1}. {instructions} ({distance})\n"
        result_txt = f'Origin: {origin}\nDestination: {destination}\nDuration: {duration_total}\nDistance: {distance_total}\nDirections:\n{directions}'

    else:
        directions = None
        distance_total = None
        duration_total = None
        result_txt = "No directions in the route requested"

    result = {'origin addr': origin, 'destination addr': destination,
              'duration': duration_total, 'distance_total': distance_total,
              'directions': directions, 'text_result': result_txt}
    return result


In [10]:
# Define GetPlacesNearby

class GetPlacesNearbyInput(BaseModel):
    address: str = Field(..., description="Address to get nearby places")
    radius: Optional[int] = Field(None, description="Radius in meters")
    keyword: Optional[str] = Field(None, description="Keyword to filter places")
    min_price: Optional[int] = Field(None, description="Minimum price level, 0: Free, 1: Inexpensive, 2: Moderate, 3: Expensive, 4: Very Expensive")
    max_price: Optional[int] = Field(None, description="Maximum price level, 0: Free, 1: Inexpensive, 2: Moderate, 3: Expensive, 4: Very Expensive")
    open_now: Optional[bool] = Field(None, description="Whether to include only places that are open now")
    rank_by: Optional[str] = Field(None, description="Ranking criteria, prominence or distance, if distance, radius canot be set")
    place_type: Optional[str] = Field(None, description="Type of the flace, e.g.: restaurant, cafe, park, museum...")
    language: str = Field(None, description="Language code (e.g., 'es', 'en')")
    limit_places: Optional[int] = Field(None, description="Number of places to return")
    google_api_key: str = Field(..., description="Google API Key")

@tool(args_schema=GetPlacesNearbyInput)
def get_places_nearby(address: str, google_api_key: str, radius: Optional[int] = 5000, keyword: Optional[str] = None,
                      min_price: Optional[int] = None, max_price: Optional[int] = None,
                      open_now: Optional[bool] = None, rank_by: Optional[str] = 'prominence',
                      place_type: Optional[str] = None, language: str = 'es', limit_places: Optional[int]=5) -> dict:
    """Get nearby places from an address"""
    # Initialize Google Maps client
    gmaps = googlemaps.Client(key=google_api_key)

    # Get the location from the provided address
    location = find_place_return_address(google_maps_API_Key, address)['candidates'][0]['geometry']['location']

    # Set up the search parameters for places_nearby
    search_params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'min_price': min_price,
        'max_price': max_price,
        'open_now': open_now,
        'rank_by': rank_by,
        'type': place_type,
        'language': language
    }

    # Remove None values to avoid sending unnecessary parameters
    search_params = {k: v for k, v in search_params.items() if v is not None}

    # Search for places nearby using Google Maps API
    places_result = gmaps.places_nearby(**search_params)

    # Extract the relevant information
    result_dict = {}
    text_results = []

    if places_result and 'results' in places_result:
        places_result['results'] = places_result['results'][:limit_places]
        for idx, place in enumerate(places_result['results']):
            name = place.get('name', 'No name available')
            address = place.get('vicinity', 'No address available')
            rating = place.get('rating', 'No rating available')
            number_reviews = place.get('user_ratings_total', 'No reviews available')
            price = place.get('price_level', 'No price level available')
            open_now_status = place.get('opening_hours', {}).get('open_now', 'No information available')
            place_id = place.get('place_id', 'No place ID available')

            # Prepare the dictionary for each place
            place_data = {
                'name': name,
                'address': address,
                'rating': rating,
                'number_reviews': number_reviews,
                'price_level': price,
                'open_now': open_now_status,
                'place_id': place_id
            }

            # Add a text summary for this place
            text_summary = (
                f"Place #{idx+1}: {name}\n"
                f"  - Address: {address}\n"
                f"  - Rating: {rating}\n"
                f"  - Number of Reviews: {number_reviews}\n"
                f"  - Price Level: {price}\n"
                f"  - Open Now: {open_now_status}\n"
                f"  - Place ID: {place_id}\n"
            )
            text_results.append(text_summary)

            # Add the place data to the result dictionary
            result_dict[f'place_{idx+1}'] = place_data

        # Add the textual result to the dictionary
        result_dict['text_result'] = "\n".join(text_results)
    else:
        result_dict['text_result'] = "No places found for the given criteria."

    return result_dict

In [12]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain.tools import tool
import googlemaps

# GetDetails
class GetDetailsInput(BaseModel):
    input_text: str = Field(..., description="Name or address of the place to get details")
    language: str = Field('es', description="Language code (e.g., 'es', 'en')")
    limit_reviews: Optional[int] = Field(5, description="Number of reviews to return")
    review_order: Optional[str] = Field(None, description="Order of the reviews: 'recent', 'high_rating', or 'low_rating'")
    google_api_key: str = Field(..., description="Google API Key")

@tool(args_schema=GetDetailsInput)
def get_details(input_text: str, google_api_key: str, language: str = 'es',
                review_order: Optional[str] = None, limit_reviews: Optional[int] = 5) -> dict:
    """Get details and reviews from a specific place."""
    # Initialize Google Maps client
    gmaps = googlemaps.Client(key=google_api_key)

    # Use find_place to search for the specific place
    # find_place_result = gmaps.find_place(input=input_text, input_type='textquery', language=language, fields=['place_id'])

    # if not find_place_result['candidates']:
    #     return {"error": "No se encontró ningún lugar con esa entrada."}

    # place_id = find_place_result['candidates'][0]['place_id']
    place_id = find_place_return_address(google_maps_API_Key, input_text=input_text)['candidates'][0]['place_id']
    # Get detailed information for the selected place
    details = gmaps.place(place_id=place_id, language=language)['result']
    # Extract reviews
    reviews = details.get('reviews', [])

    # Sort reviews based on the review_order parameter
    if review_order == 'recent':
        # Sort by most recent
        reviews = sorted(reviews, key=lambda x: x['time'], reverse=True)
    elif review_order == 'high_rating':
        # Sort by highest rating
        reviews = sorted(reviews, key=lambda x: x['rating'], reverse=True)
    elif review_order == 'low_rating':
        # Sort by lowest rating
        reviews = sorted(reviews, key=lambda x: x['rating'])

    # Extract key details
    result_dict = {
        'name': details.get('name', 'No disponible'),
        'address': details.get('formatted_address', 'No disponible'),
        'phone_number': details.get('formatted_phone_number', 'No disponible'),
        'website': details.get('website', 'No disponible'),
        'rating': details.get('rating', 'No disponible'),
        'total_reviews': details.get('user_ratings_total', 'No disponible'),
        'opening_hours': details.get('opening_hours', {}).get('weekday_text', 'No disponible'),
        'reviews': [
            {
                'author_name': review.get('author_name', 'Anónimo'),
                'rating': review.get('rating', 'No disponible'),
                'text': review.get('text', 'No hay texto de reseña'),
                'relative_time': review.get('relative_time_description', 'No disponible')
            }
            for review in reviews[:limit_reviews]  # Limit to the specified number of reviews
        ]
    }

    return result_dict


We need to format the functions that are going to be given to the model

In [13]:
functions = [format_tool_to_openai_function(t) for t in [get_coordinates, get_address_from_coordinates,
                                                         get_directions, get_places_nearby,
                                                         get_details]]

<ipython-input-13-c14649052bd6>:1: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  functions = [format_tool_to_openai_function(t) for t in [get_coordinates, get_address_from_coordinates,


## **Load the model**

In [15]:
from langdetect import detect

def detect_language(text):
    return detect(text)  # Return 'es' for spanish, 'en' for english


In [17]:
from langchain.schema.agent import AgentFinish
import json  # We import json to handle the conversion of the dictionary to a string

model = ChatOpenAI(model="gpt-3.5-turbo", api_key=OpenAI_Key, temperature=0).bind(functions=functions, function_call="auto")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant" + f"GoogleAPI_key: {google_maps_API_Key}"),
    ("user", "{input}"  f"GoogleAPI_key: {google_maps_API_Key}")
])



## **Routing part**

The routing mechanism ensures that the model first determines which function to use and what parameters are required. After executing the appropriate function and obtaining the result, the model is invoked again to generate a response tailored to the original prompt, using the retrieved data. This two-step process ensures that the output is both accurate and adapted to the language and context of the user's initial request.

In [26]:
def route(result, original_input):
    # Detect the language of the original input
    language = detect_language(original_input)

    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_coordinates": get_coordinates,
            "get_address_from_coordinates": get_address_from_coordinates,
            "get_directions": get_directions,
            "get_places_nearby": get_places_nearby,
            "get_details": get_details
        }

        # Execute the selected tool with the provided input
        tool_output = tools[result.tool].run(result.tool_input)
        # Convert the dictionary to a string in JSON format
        tool_output_str = json.dumps(tool_output)

        # Create a new prompt to generate a clear message in the correct language
        prompt2 = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant, and you will respond in the language you were spoken to."),
            ("user", f"Original question: {original_input}\nObtained data:\n{{input}}\nProvide a clear response to the original question using the obtained data clearly in {language}.")
        ])

        # Prepare the string with the new prompt
        chain2 = prompt2 | model

        # Invoke the chain with the tool output as input
        assistant_response = chain2.invoke({"input": tool_output_str})

        # Extract the assistant's message
        if hasattr(assistant_response, 'content'):
            return assistant_response.content
        else:
            return assistant_response  # In case 'content' is not available


## **Testing**

In [ ]:
from IPython.display import display, Markdown

In [1]:
input_txt = "Give me some details of the Retiro de Madrid"
# Preparamos la cadena principal, pasando el input original para mantener el contexto
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | (lambda result: route(result, input_txt))

# Invocamos la cadena con la consulta del usuario
result = chain.invoke({"input": input_txt})
display(Markdown(result))

In [2]:
input_txt = "Give me some details of what people think of the Museo Reina Sofia de Madrid"
# Preparamos la cadena principal, pasando el input original para mantener el contexto
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | (lambda result: route(result, input_txt))

# Invocamos la cadena con la consulta del usuario
result = chain.invoke({"input": input_txt})
display(Markdown(result))

In [3]:
input_txt = "Can you recommend pasta restaurants near Plaza España in Madrid?"

# Preparamos la cadena principal, pasando el input original para mantener el contexto
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | (lambda result: route(result, input_txt))

# Invocamos la cadena con la consulta del usuario
result = chain.invoke({"input": input_txt})
display(Markdown(result))

In [4]:
input_txt = "Estoy perdido en la Plaza España de Madrid y quiero ir al intercambiador de Moncloa, cuales son las indicaciones para ir en metro"


# Preparamos la cadena principal, pasando el input original para mantener el contexto
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | (lambda result: route(result, input_txt))

# Invocamos la cadena con la consulta del usuario
result = chain.invoke({"input": input_txt})
display(Markdown(result))

# **Model + agent_scratchpad**

In LangChain, the agent_scratchpad is a placeholder used to track the ongoing process of decision-making and action-taking by the language model. It logs the intermediate steps, actions, and observations that occur as the agent interacts with external systems or APIs, such as Google Maps. This feature allows the agent to maintain context and "remember" what has already been done, what actions it has attempted, and how it should proceed based on the outcomes of previous steps. By including agent_scratchpad in the prompt, LangChain ensures that this log is updated in real-time as the conversation unfolds, keeping track of the agent’s reasoning and actions.

The agent_scratchpad is especially useful in multi-step tasks where the model needs to interact with external tools, process responses, and make decisions over multiple turns. For example, when querying an API, the agent will record the query in the scratchpad, and once the response comes in, it will add that observation to the same log. This allows the model to reason based on the full history of the task, helping it decide what the next best action should be, making the conversation more dynamic and coherent over time.


In [5]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant" + f"and you know the Google API Key: {google_maps_API_Key}"),
    ("user", "{input}" ),
    MessagesPlaceholder(variable_name="agent_scratchpad")  # We are going to pass the action and the observation
])

In [35]:
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=OpenAI_Key, temperature=0).bind(functions=functions, function_call="auto")


In [36]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [45]:
result1 = chain.invoke({"input": "Obtain the coordinates of the Bernabeu in Madrid",
                        "agent_scratchpad": []})


In [7]:
result1.tool_input

In [47]:
observation = get_coordinates(result1.tool_input)

In [8]:
type(result1)

In [41]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [9]:
format_to_openai_functions([(result1, observation),])

In [10]:
result2 = chain.invoke({
    "input": "Obtain the coordinates of the Bernabeu in Madrid",
    "agent_scratchpad": format_to_openai_functions([(result1, observation),])
})
result2

The RunnablePassthrough.assign is being used here to assign a specific transformation to the agent_scratchpad. It takes the intermediate_steps from the input (x) and applies the function format_to_openai_functions to reformat these steps. This allows the agent to dynamically track and process multiple actions or function calls by reformatting the intermediate steps as the agent progresses through its tasks. Essentially, it ensures that the agent can handle multi-step processes by keeping track of its ongoing actions.

In [44]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [51]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "get_coordinates": get_coordinates,
            "get_address_from_coordinates": get_address_from_coordinates,
            "get_directions": get_directions,
            "get_places_nearby": get_places_nearby,
            "get_details": get_details
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [11]:
run_agent("Obtain the coordinates of the Bernabeu in Madrid")


In the following example we can see that the model is calling two functions

In [13]:
run_agent("Obtener las coordenadas y restaurantes cerca del Bernabeu en Madrid")

In [54]:
run_agent('Hi')

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

With the following we can follow the steps of the model

In [55]:
tools = [get_coordinates, get_address_from_coordinates, get_directions, get_places_nearby, get_details]

In [56]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({"input": "Obtain the coordinates of the Bernabeu in Madrid"})


In [15]:
agent_executor.invoke({"input": "Obtain the restaurants close to the Bernabeu in Madrid"})

But this does not include memory that is why the model cannot remember the previous message. Lets see this

In [16]:
agent_executor.invoke({"input": "Obtain the restaurants close to the place I asked you before"})

# Model + history = Chatbot

In [62]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant" + f"Remember to use the Google API key when needed: {google_maps_API_Key}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [63]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [64]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

<ipython-input-64-4554adb1bf85>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")


In [65]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [17]:
agent_executor.invoke({"input": "Obtain the coordinates of the Bernabeu in Madrid"})

In [18]:
agent_executor.invoke({"input": "Can you recommend me some restaurants here?"})

In [19]:
agent_executor.invoke({"input": "What people think about the first restaurant you recommend me?"})

In [20]:
agent_executor.invoke({"input": "Summarize"})

## Creating an **aesthetic Chatbot**

In [70]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [71]:
tools = [get_coordinates, get_address_from_coordinates, get_directions, get_places_nearby, get_details, create_your_own]

In [75]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):

    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0, api_key = OpenAI_Key).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant" + f"Remember to use the GoogleAPI when needed: {google_maps_API_Key}"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [21]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=800, width = 1000),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard